In [19]:
from sklearn.datasets import fetch_20newsgroups

In [20]:
train = fetch_20newsgroups(subset = 'train',shuffle=True)
test   = fetch_20newsgroups(subset = 'test',shuffle=True)

In [21]:
print(list(train.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [22]:
ng_train.data[:3]

["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n",
 "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 

In [23]:
print(ng_train.filenames.shape,ng_train.target.shape)

(11314,) (11314,)


In [24]:
'''
Loading gensim and nltk libraries
'''
#!pip install gensim
import gensim

# Converts into tokens (Alternative to word_tokenize)
from gensim.utils import simple_preprocess

from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
#from nltk.stem import *
import numpy as np
np.random.seed(400)

In [25]:
WordNetLemmatizer().lemmatize('runs')

'run'

In [26]:
#testing the stemming part before preprocessing. This should be changing any plural into singular word
import pandas as pd
stemmer = SnowballStemmer("english")
original_words=['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']

singles= [WordNetLemmatizer().lemmatize(plural,pos='v') for plural in original_words]

pd.DataFrame(data={'Original Words':original_words, 'Lemma':singles})

,Original Words,Lemma
0,caresses,caress
1,flies,fly
2,dies,die
3,mules,mules
4,denied,deny
5,died,die
6,agreed,agree
7,owned,own
8,humbled,humble
9,sized,size


In [27]:
# writing function for the entire dataset
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

#Tokenize and Lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
            
    return result
        

In [29]:
#preview a document before preprocessing

#document_num = 50
doc_sample = 'On Wednesday, Trump blasted the US.’s abstention from the U. N. Security Council vote on Israeli settlements earlier this month. The tweets came just hours before Secretary of State John Kerry gave a speech defending the decision and calling the continued building of settlements on Palestinian territory in the West Bank a threat to the   solution in the region. Trump’s support for Israel and Prime Minister Benjamin Netanyahu  —   who has had a fraught relationship with President Obama  —   may be the biggest forthcoming shift in immediate foreign policy between the outgoing and incoming administrations. Throughout the campaign, he pledged that his administration would be a steadfast ally of Israel. '

print("Original Document: ")
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original Document: 
['On', 'Wednesday,', 'Trump', 'blasted', 'the', 'US.’s', 'abstention', 'from', 'the', 'U.', 'N.', 'Security', 'Council', 'vote', 'on', 'Israeli', 'settlements', 'earlier', 'this', 'month.', 'The', 'tweets', 'came', 'just', 'hours', 'before', 'Secretary', 'of', 'State', 'John', 'Kerry', 'gave', 'a', 'speech', 'defending', 'the', 'decision', 'and', 'calling', 'the', 'continued', 'building', 'of', 'settlements', 'on', 'Palestinian', 'territory', 'in', 'the', 'West', 'Bank', 'a', 'threat', 'to', 'the', '', '', 'solution', 'in', 'the', 'region.', 'Trump’s', 'support', 'for', 'Israel', 'and', 'Prime', 'Minister', 'Benjamin', 'Netanyahu', '', '—', '', '', 'who', 'has', 'had', 'a', 'fraught', 'relationship', 'with', 'President', 'Obama', '', '—', '', '', 'may', 'be', 'the', 'biggest', 'forthcoming', 'shift', 'in', 'immediate', 'foreign', 'policy', 'between', 'the', 'outgoing', 'and', 'incoming', 'administrations.', 'Throughout', 'the', 'campaign,', 'he', 'pledged', 'that', 

In [30]:
processed_docs = []

for doc in train.data:
    processed_docs.append(preprocess(doc))

In [31]:
len(processed_docs)

11314

In [32]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [33]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=100000)

In [34]:
#Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
#words and how many times those words appear. Save this to 'bow_corpus'
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [35]:
#preview 
document_num = 10
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]],
                                                     bow_doc_x[i][1]))

Word 14 ("model") appears 1 time.
Word 33 ("clock") appears 1 time.
Word 46 ("keyword") appears 1 time.
Word 63 ("summari") appears 1 time.
Word 83 ("email") appears 1 time.
Word 101 ("opinion") appears 1 time.
Word 130 ("worth") appears 1 time.
Word 146 ("nice") appears 1 time.
Word 227 ("hard") appears 1 time.
Word 385 ("owner") appears 1 time.
Word 399 ("axi") appears 1 time.
Word 400 ("beemer") appears 1 time.
Word 401 ("bike") appears 2 time.
Word 402 ("brown") appears 1 time.
Word 403 ("ducati") appears 2 time.
Word 404 ("expir") appears 1 time.
Word 405 ("fade") appears 1 time.
Word 406 ("leak") appears 2 time.
Word 407 ("lonestar") appears 2 time.
Word 408 ("motor") appears 1 time.
Word 409 ("orang") appears 1 time.
Word 410 ("paint") appears 1 time.
Word 411 ("pop") appears 1 time.
Word 412 ("recommend") appears 1 time.
Word 413 ("richardson") appears 2 time.
Word 414 ("run") appears 1 time.
Word 415 ("sell") appears 1 time.
Word 416 ("shop") appears 1 time.
Word 417 ("stabl")

In [36]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [37]:
for idx, topic in lda_model.print_topics():
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.010*"bike" + 0.006*"game" + 0.005*"run" + 0.005*"team" + 0.005*"virginia" + 0.004*"motorcycl" + 0.004*"pitch" + 0.004*"player" + 0.004*"play" + 0.004*"ride"


Topic: 1 
Words: 0.010*"armenian" + 0.008*"israel" + 0.007*"isra" + 0.007*"kill" + 0.006*"turkish" + 0.005*"weapon" + 0.005*"jew" + 0.005*"govern" + 0.005*"arab" + 0.004*"gun"


Topic: 2 
Words: 0.019*"game" + 0.017*"team" + 0.013*"play" + 0.010*"player" + 0.010*"hockey" + 0.006*"season" + 0.006*"canada" + 0.005*"leagu" + 0.005*"score" + 0.005*"entri"


Topic: 3 
Words: 0.014*"window" + 0.013*"card" + 0.009*"driver" + 0.006*"video" + 0.006*"sale" + 0.006*"monitor" + 0.006*"speed" + 0.005*"appl" + 0.005*"softwar" + 0.005*"price"


Topic: 4 
Words: 0.016*"file" + 0.011*"program" + 0.010*"window" + 0.007*"imag" + 0.006*"avail" + 0.006*"data" + 0.005*"version" + 0.005*"server" + 0.005*"graphic" + 0.005*"applic"


Topic: 5 
Words: 0.018*"space" + 0.012*"nasa" + 0.007*"scienc" + 0.007*"orbit" + 0.006*"launch" + 0.005

In [38]:
num = 50
unseen_document = ng_test.data[num]
print(unseen_document)

From: carter@ecf.toronto.edu (CARTER EDWARD A)
Subject: Re: Good Reasons to Wave at each other
Organization: University of Toronto, Engineering Computing Facility
Lines: 19

jlevine@rd.hydro.on.ca (Jody Levine) writes:
>Has anyone, while driving a cage, ever waved at bikers? I get the urge,
>but I've never actually done it.

Oh yeah, all the time.  On a nice spring/summer day, I roll down the window
and drive around looking for bikes.  When a bike motors by in the opposite
direction, I stick my arm out and hi5'em.  My arm feels like a million 
bucks when I'm doing this a 60km/h.  I do the same thing with cyclists.
The only problem with hi5ing a cyclist is their always in the right hand lane.
I hafta roll down the other window and hi5 them on the back.  Oh well, I 
think they appreciate the thought. 

Regards, Ted.

---
University of Toronto Computer Engineering               
PowerUsersGroupChairman
'89 FZR600: I'm taking a ride with my best friend.                  DoD#:886699




In [39]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 30)))

Score: 0.6837900280952454	 Topic: 0.010*"bike" + 0.006*"game" + 0.005*"run" + 0.005*"team" + 0.005*"virginia" + 0.004*"motorcycl" + 0.004*"pitch" + 0.004*"player" + 0.004*"play" + 0.004*"ride" + 0.004*"leav" + 0.003*"basebal" + 0.003*"duke" + 0.003*"dave" + 0.003*"cramer" + 0.003*"defens" + 0.003*"homosexu" + 0.003*"rider" + 0.003*"smith" + 0.003*"rid" + 0.003*"home" + 0.003*"david" + 0.003*"season" + 0.003*"gatech" + 0.003*"base" + 0.003*"hand" + 0.003*"optilink" + 0.003*"lose" + 0.003*"mark" + 0.003*"littl"
Score: 0.2451610267162323	 Topic: 0.014*"window" + 0.013*"card" + 0.009*"driver" + 0.006*"video" + 0.006*"sale" + 0.006*"monitor" + 0.006*"speed" + 0.005*"appl" + 0.005*"softwar" + 0.005*"price" + 0.005*"memori" + 0.004*"machin" + 0.004*"version" + 0.004*"mous" + 0.004*"engin" + 0.004*"cwru" + 0.004*"color" + 0.004*"printer" + 0.004*"modem" + 0.004*"sell" + 0.003*"cleveland" + 0.003*"simm" + 0.003*"board" + 0.003*"port" + 0.003*"instal" + 0.003*"upgrad" + 0.003*"access" + 0.003*"s